# Baseline LP Generator


In [ ]:
# Copyright (C) to Yingcong Tan, Andrew Delong, Daria Terekhov. All Rights Reserved.

import numpy as np
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
import deep_inv_opt as io
import os
    
# In this notebook, a set of baseline lps will be gerenated.
# All generated files will be saved in the following directory, you might override it.
PATH = "C:/Users/Public/Downloads/"


### Utility Functions
**def feasibility**: check feasibility using constraints  
**test_LP** : check if the random generated LP is feasible
solve linprog (A,b, eps = 1e-5), if sol is found, the generated LP is feasible   
**plot_LP**: plot LPs for 2D case with all target potins labeled

In [ ]:
def feasibility(point, A_ub, b_ub, tolerance= 6):
    m,n = A_ub.shape
    feasibility_ub = np.round(A_ub @ point.reshape((n,1)) - b_ub.reshape((m,1)), tolerance)
    if (feasibility_ub <=0).all():
        return "feasible"
    elif (feasibility_ub >0).any():
        return "infeasible"  
        
def test_LP (A, b):
    """
    solve the generated linear program with a random c vector
            min c'x 
            st. A x <= b,     
    """
    m, n = A.shape
    c = np.random.normal(0,1, n)
    c_vector = io.tensor(c).view(-1,1)
    A_ub = io.tensor(A)
    b_ub = io.tensor(b).view(-1,1)

    sol  = io.linprog (c_vector, A_ub, b_ub, eps = 1e-7).numpy()
    if sol.size>0:
        return True, sol
    elif sol.size ==0:
        return False, sol
        
### plot lp only for 2D instance
def plot_LP(filename, num, hull, points, 
            opt_points, fea_points=None, infea_points=None):
    # plot feasible region in 2D
    m,n = opt_points.shape
    plt.figure(figsize=(5,5))    
    for simplex in hull.simplices:
        plt.plot(points[simplex, 0], points[simplex, 1], 'k-', linewidth=2)
    plt.plot(opt_points[:,0], opt_points[:,1], 'go', 
             label='Opt sol',markersize=12, mfc="None", mew = 2)

    
    for ind_target in range(len(opt_points)):
        plt.text(opt_points[ind_target, 0] + 0.03, opt_points[ind_target, 1] - 0.05, 
                 "Vertex Pt%d" % (ind_target+1), color='g', fontsize = 10)
    
    if fea_points is not None:
        for ind_target in range(len(fea_points)):
            plt.text(fea_points[ind_target, 0] + 0.03, fea_points[ind_target, 1] - 0.05, 
                     "Fea. Pt%d" % (ind_target+1), color='b', fontsize = 10)
        plt.plot(fea_points[:,0], fea_points[:,1], 'b+', 
                 label='Fea sol',markersize=12, mew = 2)

    if infea_points is not None:
        for ind_target in range(len(infea_points)):
            plt.text(infea_points[ind_target, 0] + 0.03, infea_points[ind_target, 1] - 0.05, 
                     "Infea. Pt%d" % (ind_target+1), color='r', fontsize = 10)   
        plt.plot(infea_points[:,0], infea_points[:,1], 'rx', 
                 label = 'Infea sol',markersize=12, mew = 2)
    plt.axis("equal")
    plt.xlabel('X1', fontsize=10)
    plt.ylabel('X2', fontsize=10)
    plt.title("Instance %s" % (num), fontsize =8)    
    plt.legend(fontsize=10)
    plt.savefig(filename+'.jpg', format='jpeg', framon = True,bbox_inches='tight', dpi=100)
    plt.show()

- **record_LP**, record the random LP instances in txt file.
- **readLP**, read LP information from the txt file.

In [ ]:
def record_LP(filename, A, b, vertex):
    """
    Save LP instance in txt file in the following format:
    m,n  # num of rows and columns
    A_ub
    b_ub
    vertex
    feasible points    (set to None in default)
    infeasible points  (set to None in default)
    """
    m,n = A.shape
    with open(filename, 'w') as LP_file:
        np.savetxt(LP_file, np.array(A.shape).reshape((1,2)), fmt="%.d")
        LP_file.write("\n")

        LP_file.write("A_ub\n")
        np.savetxt(LP_file, A, fmt="%.6f")
        LP_file.write("\n")
        LP_file.write("b_ub\n")
        np.savetxt(LP_file, b, fmt="%.6f")
        LP_file.write("\n")        
        
        LP_file.write("%d vertices\n"% len(vertex) )
        np.savetxt(LP_file, vertex, fmt="%.6f")
        LP_file.write("\n")

### LP Generator (numInstances, n, nCons, numPoints, vertex, callback)
- **numInstances**, number of LP instances to generate  
- **(n, nCons)**, LP characteristics, n - num of varialbes, nCons - number of constraints
- **numPoints**, number of points to generate LP through **ConvexHull** function 
- **vertex = True**, Ture or False, whether or not generate vertex points
- **num_fea = 0**, # of feasible points to generate
- **num_infea = 0**, # of infeasible points to generate
- **callback = plot_LP**, plotting function, which only works for 2D LP

In [ ]:
def LPGenerator(numInstances, n, nCons, numPoints,
                vertex = True,
                callback = plot_LP):
    ind_lp = 1
    while ind_lp <=numInstances: 
        points = np.random.normal(0,1, (numPoints, n)) 
        # Note that this means there will be *AT MOST* n constraints
        hull = ConvexHull(points) 
        m,numVars = hull.equations.shape
        if m == nCons:
            assert numVars-1 == n, "Number of variables after convex hull function not equal to the original number of vars"

            A = -1 * hull.equations[:,0:n]
            b = hull.equations[:,n]

            print("---------------- Instance %d ----------------"%(ind_lp))

            # important: the convexhull generates Ax >= b (not Ax <= b as linprog requires)
            # convert A and b s.t. the constraints have the form as: Ax <= b
            A = np.round(np.negative(A),10)
            b = np.round(np.negative(b),10)

            print("A_ub size ", A.shape, "\n")
            print("b_ub size ", b.shape, "\n")

            # Proceed if the lp instance is feasible
            flag, _, = test_LP (A, b)
            if flag == True:
                # Generate points
                # vertices == use built in function
                if vertex == True:
                    opt_points = points[hull.vertices,:] 
                print("%d vertices\n"%opt_points.shape[0])



                print()
                                
                directory = PATH + "Deep_Inv_Opt/LP baseline/n=%d,m=%d/"%(n, m)
                if not os.path.exists(directory):
                    os.makedirs(directory)
                filename = directory + "%svar_LP%s.txt"%(str(n).zfill(2), str(ind_lp).zfill(2))
                record_LP(filename, A, b, opt_points)

                # plot the
                if callback and n <=2:
                    callback(filename[:-4], ind_lp, hull, points, opt_points)

                ind_lp+=1
                print("--------------------------------------")
                print()


### Run the following cell to generate complete test instances to be used in other notebooks:
- Figure 3_step2_(i)_Learn c
- Figure 3_step3_(ii)_Learn A,b,c jointly
- Figure 4_Parametric LP

#### Note, we did not fix the seed in the random lp generation process in our paper.  Thus, the baseline LPs generated here won't be identical to our paper. 

In [ ]:
# number of variables
var_range = [2,10]
# number of constraints: the first row corresponds to 2D lp instances
# and the second row corresponds to 10D lp instances
cons_range = [[4,  8,  16],
              [20, 36, 80]]

# number of random generated point which are then used for constructing convexhull using scipy.spatial.ConvexHull function
# these are suggested values for efficient LP generation, you might also use other values
numPoints = [[5,  8,  150], 
             [12, 12, 13]]

# number of random lp instances to generate for different size
num_ins = 50

for i, nVar in enumerate (var_range):
    for j, nCons in enumerate (cons_range[i]):
        num_p = numPoints[i][j]
        LPGenerator(num_ins, nVar, nCons, num_p)